In [1]:
# !pip install zodipy
import astropy.units as u
import zodipy
import numpy as np

In [ ]:
from astropy.coordinates import SkyCoord, get_sun
from astropy.time import Time
import astropy.units as u

def pointing_geometry(ra_deg, dec_deg, obstime):
    """
    Compute heliocentric elongation and ecliptic latitude of a target.

    Parameters
    ----------
    ra_deg (float): Target Right Ascension in degrees (ICRS).
    dec_deg (float): Target Declination in degrees (ICRS).
    obstime (str or astropy.time.Time): Observation time (e.g., "2025-09-24T00:00:00").

    Returns
    -------
    elongation (float): Heliocentric elongation (deg) = angular separation between Sun and target along ecliptic.
    beta (float): Heliocentric ecliptic latitude (deg) of target.
    """

    if not isinstance(obstime, Time):  # convert to astropy Time if needed
        obstime = Time(obstime)

    # Target in ICRS
    target = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg, frame="icrs")

    # Sun position at obstime
    sun_icrs = get_sun(obstime)

    # print(target, sun_icrs)
    # Convert Solar icrs position to RA/Dec
    print("Solar RA, Dec:", sun_icrs.ra.hour, sun_icrs.dec.deg)
    print("Target RA, Dec:", target.ra.hour, target.dec)

    # Convert both to true ecliptic coordinates
    target_ecl = target.barycentrictrueecliptic
    sun_ecl = sun_icrs.barycentrictrueecliptic

    # Longitude difference (wrap to [0, 180])
    delta_lon = abs((target_ecl.lon - sun_ecl.lon).wrap_at(180*u.deg))
    elongation = delta_lon.deg

    # Ecliptic latitude of target
    beta = abs(target_ecl.lat.deg)

    return elongation, beta


In [45]:
print("Current Time (UTC):", Time.now().iso)

elong, beta = pointing_geometry(ra_deg=230, dec_deg=0, obstime=Time.now())

print(f"\nElongation = {elong:.2f} deg, Beta = {beta:.2f} deg")


Current Time (UTC): 2025-09-23 20:56:23.345
Solar RA, Dec: 12.044383499168731 -0.28835582196880455
Target RA, Dec: 15.333333333333336 0.0

Elongation = 7.88 deg, Beta = 17.74 deg


In [ ]:
# For a specific wavelength (e.g. 25 μm, in the mid-IR)
model = zodipy.Model(25 * u.micron)

# Or define a bandpass:
points = [3, 4, 5, 6] * u.micron
weights = [0.2, 0.4, 0.3, 0.1]
model = zodipy.Model(points, weights=weights)